In [1]:
## ANA

import json
from pprint import pprint

f = open("areacliente.repsol.es.har", "r")
datoshorarios = f.read()
f.close()

datoshorarios = json.loads(datoshorarios)


lista = []
fechasunicas = set()

for entrada in datoshorarios["log"]["entries"]:
    try:
        url:str = entrada["request"]["url"]
        if url.startswith("https://pro-wally-back.cloudapp.repsol.com/v2/houses/a4j4I000002VQw6QAG/products/4302431903/consumption/hourly?"):
            fecha = url[-8:]
            if fecha not in fechasunicas:
                datitos:str = entrada["response"]["content"]["text"]
                datitos = json.loads(datitos)
                for tiempo in datitos:
                    consumo = tiempo["consumption"]
                    hora = tiempo["hour"]
                    diccionario = {"fecha": fecha,
                                "hora": hora,
                                "consumo": consumo}
                    lista.append(diccionario)
                    fechasunicas.add(fecha)
    except KeyError:
        pass

valle = "valle"
punta = "punta"
llano = "llano"

# fecha 20230731
# 1 de julio 2023 fue sábado
# ojo revisar duplicados
# De 10h a 14h y de 18h a 22h Período Punta (P1)
# De 8h a 10h, de 14h a 18h y de 22h a 24h Período Llano (P2)
# De 00h a 8h (de lunes a viernes) y las 24h (fines de semana y festivos) Período Valle (P3)

discriminación = {
    "00": valle,
    "01": valle,
    "02": valle,
    "03": valle, 
    "04": valle,
    "05": valle,
    "06": valle,
    "07": valle,
    "08": llano,
    "09": llano,
    "10": punta,
    "11": punta,
    "12": punta,
    "13": punta,
    "14": llano,
    "15": llano,
    "16": llano,
    "17": llano,
    "18": punta, 
    "19": punta,
    "20": punta, 
    "21": punta,
    "22": llano,
    "23": llano,
                }

for diccionario in lista:
    
    diasem = int(diccionario["fecha"]) % 7
    # Para no pasar a fechas, calculamos el módulo de 202307XX / 7. El 1 de julio fue sábado.
    if diasem == 1 or diasem == 2:
        tipo = valle
    else:
        hora = diccionario["hora"][:2]
        tipo = discriminación[hora]    
    diccionario["tipo"] = tipo

# consumo_punta = 0
# consumo_valle = 0
# consumo_llano = 0
# for diccionario in lista:
    
#     if diccionario["tipo"] == punta:
#         consumo_punta = consumo_punta + float(diccionario["consumo"])
#     if diccionario["tipo"] == valle:
#         consumo_valle = consumo_valle + float(diccionario["consumo"])
#     if diccionario["tipo"] == llano:
#         consumo_llano = consumo_llano + float(diccionario["consumo"])

consumos = {punta:0,
            llano:0,
            valle:0}

for diccionario in lista:
    tipo = diccionario["tipo"]
    consumos[tipo] = consumos[tipo] + float(diccionario["consumo"])

#pprint(lista)
#pprint ([consumo_punta, consumo_llano, consumo_valle])
#[17.999999999999996, 102.05999999999985, 19.450000000000003]
pprint(consumos)

### --------COMPARATIVA LUZ--------###

# Julio

dias:int = 31
potencia = 3.5 #kW
excedentes = 374.32 #kWh
iva = 1.21
bono_social = 0.006282 #€/día
impuesto = 3.8/100
equipos = 0.82 #€

octopus = {
    "nombre": "Octopus",
    "punta": 0.1780, #€/kWh
    "llano": 0.1300, #€/kWh
    "valle": 0.0810, #€/kWh
    "P1": 0.089, #€/kW/día
    "P2": 0.026, #€/kW/día
    "excedente": 0.05, #€/kWh
    "batería": 0
}

repsol_solar = {
    "nombre": "Repsol solar",
    "punta": 0.149746, #€/kWh
    "llano": 0.149746, #€/kWh
    "valle": 0.149746, #€/kWh
    "P1": 0.068219, #€/kW/día
    "P2": 0.068219, #€/kW/día
    "excedente": 0.10, #€/kWh
    "batería": 0
}


repsol_bateria = {
    "nombre": "Repsol bateria",
    "punta": 0.139746, #€/kWh
    "llano": 0.139746, #€/kWh
    "valle": 0.139746, #€/kWh
    "P1": 0.068219, #€/kW/día
    "P2": 0.068219, #€/kW/día
    "excedente": 0.08, #€/kWh
    "batería": 1.99/30
}

repsol_old = {
    "nombre": "Repsol old",
    "punta": 0.170589, #€/kWh
    "llano": 0.170589, #€/kWh
    "valle": 0.170589, #€/kWh
    "P1": 0.069190567, #€/kW/día
    "P2": 0.068128167, #€/kW/día
    "excedente": 0.1499, #€/kWh
    "batería": 1.99/30
}

naturgy = {
    "nombre": "Naturgy",
    "punta": 0.17, #€/kWh
    "llano": 0.1162, #€/kWh
    "valle": 0.0821, #€/kWh
    "P1": 0.103663, #€/kW/día
    "P2": 0.034042, #€/kW/día
    "excedente": 0.11, #€/kWh
    "batería": 1.80/30
}

tarifas = [
    octopus,
    repsol_solar,
    repsol_bateria,
    repsol_old,
    naturgy
]

comparativa = []
for empresa in tarifas:
    consumo_total = empresa["punta"]*consumos[punta]+empresa["llano"]*consumos[llano]+empresa["valle"]*consumos[valle]
    potencia_total = potencia*dias*(empresa["P1"]+empresa["P2"])
    excedentes_total = excedentes*empresa["excedente"]
    precio = ((potencia_total+bono_social*dias+consumo_total)*(1+impuesto)+equipos+empresa["batería"]*dias)*iva-excedentes_total
    calculado = {
        "Empresa": empresa["nombre"],
        "Precio": precio,
                 }
    comparativa.append(calculado)

pprint(comparativa)

FileNotFoundError: [Errno 2] No such file or directory: 'areacliente.repsol.es.har'